In [23]:
import json
import cv2
import os
from glob import glob
from sklearn.model_selection import train_test_split
ann_path = './custom_dataset/train/*.txt'
ann_files = glob(ann_path)
# valid set을 나눌 경우
ann_train, ann_valid = train_test_split(ann_files,
                                       test_size=0.1,
                                       random_state=1119)
def labelmetxt2coco(anns):
    cars = {}
    # 숫자로 해도 되지만, 클래스 확인하기 위해 차종으로 변경
    classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", 
               "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", 
               "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", 
               "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", 
               "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", 
               "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", 
               "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
               "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", 
               "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
               "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", 
               "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", 
               "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", 
               "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", 
               "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", 
               "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", 
               "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", 
               "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]
    cars["categories"] = [{"id":i,"name":cat,"supercategory":"none"} for i, cat in enumerate(classes)]
    cars["images"] = []
    cars["annotations"] = []
    cnt_ann = 0
    for i, ann in enumerate(anns):
        img_path = ann.replace("txt", "png")
        #h, w, _ = cv2.imread(img_path).shape
        cars["images"].append({"id":i,"height":1040,"width":1920,"file_name":img_path})

        f_ann = open(ann, "r")
        for line in f_ann.readlines():
            data = line.split()
            cat = int(float(data[0]))
            pt1x = int(data[1])
            pt1y = int(data[2])
            #pt2x = int(data[3])    not to use
            #pt2y = int(data[4])
            pt3x = int(data[5])
            pt3y = int(data[6])
            #pt4x = int(data[7])
            #pt4y = int(data[8])
            x = pt1x
            y = pt1y
            width = pt3x - pt1x
            height = pt3y - pt1y
            area = width * height
            cars["annotations"].append({"id": cnt_ann,
                                        "image_id": i,
                                        "category_id": cat,
                                        "bbox": [x, y, width, height],
                                        "area": area,
                                        "segmentation": [],
                                        "iscrowd": 0})
            cnt_ann += 1
        
    return cars

# with open('/content/drive/MyDrive/DACON/236107/data/annotations/train_json', "w") as f:
#     json.dump(labelmetxt2coco(ann_files), f, ensure_ascii=False, indent=4)

# # train/valid split을 했을 경우
# with open('./custom_dataset/annotations/train.json', "w") as f:
#     json.dump(labelmetxt2coco(ann_train), f, ensure_ascii=False, indent=4)

# with open('./custom_dataset/annotations/val.json', "w") as f:
#     json.dump(labelmetxt2coco(ann_valid), f, ensure_ascii=False, indent=4)

def labelmetxt2cocotest(anns):
    cars = {}
    # 숫자로 해도 되지만, 클래스 확인하기 위해 차종으로 변경
    classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", 
               "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", 
               "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", 
               "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", 
               "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", 
               "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", 
               "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
               "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", 
               "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
               "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", 
               "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", 
               "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", 
               "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", 
               "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", 
               "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", 
               "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", 
               "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]
    cars["categories"] = [{"id":i,"name":cat,"supercategory":"none"} for i, cat in enumerate(classes)]
    cars["images"] = []
    cars["annotations"] = []
    cnt_ann = 0
    for i, ann in enumerate(anns):
        cars["images"].append({"id":i,"height":1040,"width":1920,"file_name":ann})
        cars["annotations"].append({"id": cnt_ann,
                                    "image_id": i,
                                    "category_id": 0,
                                    "bbox": [0, 0, 0, 0],
                                    "area": 0,
                                    "segmentation": [],
                                    "iscrowd": 0})
        cnt_ann += 1
        
    return cars
ann_path = './custom_dataset/test/*.png'
ann_test = glob(ann_path)

# with open('./custom_dataset/annotations/test.json', "w") as f:
#     json.dump(labelmetxt2cocotest(ann_test), f, ensure_ascii=False, indent=4)

In [ ]:
%run tools/train.py configs/faster_rcnn/faster_rcnn_x101_64x4d_fpn_1x_cars.py

In [1]:
%run tools/test.py work_dirs/faster_rcnn_x101_64x4d_fpn_1x_cars/faster_rcnn_x101_64x4d_fpn_1x_cars.py work_dirs/faster_rcnn_x101_64x4d_fpn_1x_cars/epoch_12.pth --out work_dirs/faster_rcnn_x101_64x4d_fpn_1x_cars/result.pkl --format-only

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
load checkpoint from local path: work_dirs/faster_rcnn_x101_64x4d_fpn_1x_cars/epoch_12.pth
[                                                  ] 0/3400, elapsed: 0s, ETA:
[                               ] 1/3400, 0.1 task/s, elapsed: 12s, ETA: 41133s
[                               ] 2/3400, 0.2 task/s, elapsed: 12s, ETA: 20926s
[                               ] 3/3400, 0.2 task/s, elapsed: 12s, ETA: 14092s
[                               ] 4/3400, 0.3 task/s, elapsed: 13s, ETA: 10679s
[                               ] 5/3400, 0.4 task/s, elapsed: 13s, ETA:  8628s
[                               ] 6/3400, 0.5 task/s, elapsed: 13s, ETA:  7282s
[                               ] 7/3400, 0.5 task/s, elapsed: 13s, ETA:  6342s
[                               ] 8/3400, 0.6 task/s, elapsed: 13s, ETA:  5629s
[                               ] 9/3400, 0.7 task/s, elapsed: 13s, ETA:  5078s
[                          

c:\Python\lib\site-packages\mmcv\__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
c:\users\kdy\downloads\mmdetection-2.28.2\mmdet\utils\setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
c:\users\kdy\downloads\mmdetection-2.28.2\mmdet\utils\setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to a

In [2]:
import pickle
import pandas as pd
import glob
with open("./work_dirs/faster_rcnn_x101_64x4d_fpn_1x_cars/result.pkl","rb") as fr:
    data = pickle.load(fr)
                                             
test_img_paths = sorted(glob.glob('./custom_dataset/test/*.png'))                                                                                                              

results = pd.read_csv('./custom_dataset/sample_submission.csv')                 
for idx in range(len(test_img_paths)):                                                                            
  prediction = data[idx]                 
  #img = cv2.imread(test_img_paths[idx])
  filename = test_img_paths[idx].split('/')[-1].split('\\')[-1]             
  for i in range(34):
    if prediction[i].shape[0]!=0:               
      label = i                    
      for j in range(prediction[i].shape[0]):
        x1, y1, x2, y2 = prediction[i][j][0], prediction[i][j][1], prediction[i][j][2],prediction[i][j][3]
        score = prediction[i][j][4]                
        
        results = results.append({
            "file_name" : filename,               
            "class_id" : label,                                                        
            "confidence" : score,                     
            "point1_x" : x1, "point1_y" : y1,
            "point2_x" : x2, "point2_y" : y1,
            "point3_x" : x2, "point3_y" : y2,
            "point4_x" : x1, "point4_y" : y2,
        }, ignore_index = True)
    

In [3]:
results.to_csv('submit.csv', index = False)

In [ ]:
import os
import cv2
pred = pd.read_csv('Results/2.csv')
img_path = './custom_dataset/test'
classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", 
               "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", 
               "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", 
               "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", 
               "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", 
               "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", 
               "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
               "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", 
               "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
               "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", 
               "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", 
               "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", 
               "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", 
               "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", 
               "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", 
               "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", 
               "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]

for i in range(len(pred)):
    scores = pred.iloc[i, :]
    label = "{}".format(classes[scores[1]]) + ' ' + str(scores[2])
    image_path = os.getcwd()+'/custom_dataset/test/'+scores[0]
    image = cv2.imread(image_path)
    cv2.rectangle(image, (int(scores[3]), int(scores[4])), (int(scores[7]), int(scores[8])), (0, 255, 0), 2)
    cv2.putText(image, label, (int(scores[3]), int(scores[4])-4), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        2 * 0.75, (0, 255, 0), 2, lineType=cv2.LINE_AA)
    cv2.imshow("Cars", image)
    cv2.waitKey()
    cv2.destroyAllWindows()